In [1]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import grad

# device = torch.device('cuda')
device = torch.device('cpu')

sys.path.append('utils_discrete')
from utils_discrete import preprocess_data_discrete_inference, plot_results_discrete_inference

In [4]:
# Load the dataset and preprocess it
data_path = 'data/burgers_shock.mat'

N = 250
q = 500
noise = 0.0
idx_t0 = 10
idx_t1 = 90

x, t, u_exact, T, lb, ub, dt, x0, u0, x1, test_X, test_u, IRK_weights = preprocess_data_discrete_inference(data_path, idx_t0, idx_t1, q, N, noise)

x0 = torch.tensor(x0, dtype = torch.float, requires_grad = True, device = device)
u0 = torch.tensor(u0, dtype = torch.float, requires_grad = True, device = device)
x1 = torch.tensor(x1, dtype = torch.float, requires_grad = True, device = device)
test_X = torch.tensor(test_X, dtype = torch.float, requires_grad = True, device = device)
test_u = torch.tensor(test_u, dtype = torch.float, requires_grad = True, device = device)
dt = torch.tensor(dt, dtype = torch.float, requires_grad = True, device = device)
IRK_weights = torch.tensor(IRK_weights, dtype = torch.float, requires_grad = True, device = device)

In [5]:
class NeuralNet(nn.Module):

    def __init__(self, layers, lb, ub):
        super().__init__()
        self.lb = torch.tensor(lb, dtype = torch.float, device = device)
        self.ub = torch.tensor(ub, dtype = torch.float, device = device)
        
        # Layer module
        self.layers = nn.ModuleList()
        
        # Make the neural network
        input_dim = layers[0]
        for output_dim in layers[1:]:
            self.layers.append(nn.Linear(input_dim, output_dim))
            nn.init.xavier_normal_(self.layers[-1].weight)
            input_dim = output_dim
        
        
    def forward(self, X):
        x = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for layer in self.layers[:-1]:
            x = torch.tanh(layer(x))

        outputs = self.layers[-1](x)
        return outputs

In [6]:
class PINN:
    def __init__(self, dt, x0, u0, x1, lb, ub, IRK_weights, nu, layers = [2, 20, 20, 20, 1], lr = 1e-2, device = torch.device('cpu')):
        self.nu = nu
        
        self.dt = dt
        self.x0 = x0
        self.u0 = u0
        self.x1 = x1
        self.IRK_weights = IRK_weights
        
        self.net_u = NeuralNet(layers, lb, ub)
        self.net_u.to(device)
        self.optimizer = torch.optim.Adam(self.net_u.parameters(), lr = lr, betas=(0.9, 0.999))
    
    def forward_gradients(self, y, x):
        # x.shape -> (#data_points x 1) (250 x 1)
        # y.shape -> (#data_points x #RK stages) (250 x 500)
        temp1 = torch.ones(y.size(), device = device, requires_grad = True)
        temp2 = torch.ones(x.size(), device = device)
        
        g = grad(y, x, grad_outputs = temp1, create_graph = True)[0]
        dy = grad(g, temp1, grad_outputs = temp2, create_graph = True)[0]
        
        del temp1, temp2
        
        # Returns a (#data_points x #RK stages) matrix (250 x 500)
        return dy
    
    def net_u0(self):
        
        u1 = self.net_u(self.x0)
        u = u1[:,:-1]
        
        u_x  = self.forward_gradients(u, self.x0)
        u_xx = self.forward_gradients(u_x, self.x0)
        
        
        f = - u*u_x + (self.nu)*u_xx
        
        u0 = u1 - self.dt*torch.matmul(f,self.IRK_weights.T)
        
        return u0
    
    def net_u1(self):
        return self.net_u(self.x1)
    
    def loss_f(self, u0, u0_pred):
        return torch.mean(torch.square(u0-u0_pred)) + torch.mean(torch.square(self.net_u1()))
    
    def optimizer_step(self):
        # Compute losses

        # Zero the grads for the model paramters in the optimizer
        self.optimizer.zero_grad()
            
        # Compute the losses and backpropagate the losses
        loss = self.loss_f(self.u0, self.net_u0())
        loss.backward()

        # Optimizer one iteration with the given optimizer
        self.optimizer.step()
        
        return loss.item()
    
    def fit(self, epochs = 1):
        for epoch in range(epochs):
            loss_value = self.optimizer_step()

            if epoch % 100 == 99:
                with torch.no_grad():
                    valid_loss = torch.norm(test_u - pinn.predict(test_X)[:,-1], 2)/torch.norm(test_u, 2)
                    print(f'Epoch {epoch+1}: Training Loss = {loss_value}, Validation Loss = {valid_loss.item()}')
    
    def predict(self, test_X):
        return self.net_u(test_X)

In [7]:
pinn = PINN(dt, x0, u0, x1, lb, ub, IRK_weights, nu = (0.01/np.pi), layers = [1, 50, 50, 50, q+1], lr = 1e-3)

In [8]:
pinn.fit(epochs = 3000)

Epoch 100: Training Loss = 0.38780128955841064, Validation Loss = 0.7980650067329407
Epoch 200: Training Loss = 0.06589822471141815, Validation Loss = 0.6330591440200806
Epoch 300: Training Loss = 0.049171850085258484, Validation Loss = 0.5695068836212158
Epoch 400: Training Loss = 0.029959149658679962, Validation Loss = 0.5061505436897278
Epoch 500: Training Loss = 0.01575472764670849, Validation Loss = 0.37142205238342285
Epoch 600: Training Loss = 0.01085569430142641, Validation Loss = 0.2782757580280304
Epoch 700: Training Loss = 0.008294481784105301, Validation Loss = 0.21886804699897766
Epoch 800: Training Loss = 0.006762710865586996, Validation Loss = 0.18273207545280457
Epoch 900: Training Loss = 0.0054414840415120125, Validation Loss = 0.15740396082401276
Epoch 1000: Training Loss = 0.00444675050675869, Validation Loss = 0.14091873168945312
Epoch 1100: Training Loss = 0.0036396037321537733, Validation Loss = 0.1250518411397934
Epoch 1200: Training Loss = 0.003010532818734646, 

In [9]:
# Evaluate the model
u1_pred = pinn.predict(test_X)

valid_loss = torch.norm(test_u - u1_pred[:,-1], 2)/torch.norm(test_u, 2)
print(f'Validation Loss: {valid_loss.item()}')

Validation Loss: 0.04136437922716141


In [10]:
# Plot the results
%matplotlib widget
device = torch.device('cpu')

plot_results_discrete_inference(x, t, x0.to(device).detach().numpy(), u0.to(device).detach().numpy(), u_exact,
                                  test_X.to(device).detach().numpy(), u1_pred.to(device).detach().numpy(),
                                    idx_t0, idx_t1, lb, ub)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …